# Building Machine Learning Classifiers: Building a basic Random Forest model

### Read in & clean text

In [61]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())], axis=1)

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,

### Explore RandomForestClassifier through Cross-Validation

In [30]:
from sklearn.model_selection import KFold, cross_val_score


In [48]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.96947935, 0.97307002, 0.97394429, 0.96675651, 0.97214735])

# Explore RandomForestClassifier through Holdout Set

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [49]:
X_train,X_test,y_train,y_test = train_test_split(X_features,data['label'],test_size=0.2)
print(len(X_train))
print(len(X_test))
print(len(X_features))

4453
1114
5567


In [50]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs=1)
rf_model = rf.fit(X_train,y_train)

In [57]:
rf_model.feature_importances_

array([0.07428093, 0.00490723, 0.00019022, ..., 0.00039936, 0.        ,
       0.        ])

In [35]:
sorted(zip(rf_model.feature_importances_,X_train.columns),reverse=True)[0:10]

[(0.07221251664413995, 'body_len'),
 (0.07025612911752044, 7350),
 (0.03687599228705966, 4796),
 (0.03518843291004981, 2031),
 (0.03197579269485929, 1803),
 (0.025831203389402334, 3134),
 (0.018301001942004097, 5988),
 (0.017960804132131705, 6285),
 (0.015037661564691981, 5724),
 (0.014337512949376607, 6746)]

In [36]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [37]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.657 / Accuracy: 0.959


# Build our own Grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

In [38]:
def train_RF(n_est,depth):
    rf = RandomForestClassifier(n_estimators=n_est,max_depth=depth,n_jobs=-1)
    rf_model = rf.fit(X_train,y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est :{}/ Depth: {},-------Precesion: {}/ Recall: {}/ Accuracy: {}'.format(n_est,depth,round(precision,3),round(recall,3),
                                                                                    round((y_pred==y_test).sum()/len(y_pred),3)))

In [39]:
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        train_RF(n_est,depth)

Est :10/ Depth: 10,-------Precesion: 1.0/ Recall: 0.313/ Accuracy: 0.917
Est :10/ Depth: 20,-------Precesion: 1.0/ Recall: 0.619/ Accuracy: 0.954
Est :10/ Depth: 30,-------Precesion: 1.0/ Recall: 0.739/ Accuracy: 0.969
Est :10/ Depth: None,-------Precesion: 0.991/ Recall: 0.791/ Accuracy: 0.974
Est :50/ Depth: 10,-------Precesion: 1.0/ Recall: 0.269/ Accuracy: 0.912
Est :50/ Depth: 20,-------Precesion: 1.0/ Recall: 0.597/ Accuracy: 0.952
Est :50/ Depth: 30,-------Precesion: 1.0/ Recall: 0.754/ Accuracy: 0.97
Est :50/ Depth: None,-------Precesion: 1.0/ Recall: 0.858/ Accuracy: 0.983
Est :100/ Depth: 10,-------Precesion: 1.0/ Recall: 0.306/ Accuracy: 0.917
Est :100/ Depth: 20,-------Precesion: 1.0/ Recall: 0.567/ Accuracy: 0.948
Est :100/ Depth: 30,-------Precesion: 1.0/ Recall: 0.761/ Accuracy: 0.971
Est :100/ Depth: None,-------Precesion: 0.991/ Recall: 0.858/ Accuracy: 0.982


# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV


**Grid-search:**: Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation**: Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used 
as the holdout set in each iteration.

In [41]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [45]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(x_tifdf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,64.443323,0.793377,0.708498,0.104827,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978475,0.977538,0.974843,...,0.974672,0.003794,1,0.999326,0.999102,0.999102,0.999775,0.999326,0.999326,0.000246
11,50.151165,0.911049,0.483887,0.023117,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979372,0.975741,0.974843,...,0.973594,0.004053,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
10,33.344873,2.530221,0.464649,0.094898,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.977538,0.973944,...,0.973415,0.003396,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
5,50.365459,1.139707,0.584306,0.038363,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.977578,0.973046,0.973944,...,0.972876,0.002988,4,0.994385,0.992366,0.995510,0.993938,0.992366,0.993713,0.001213
7,32.434976,1.207455,0.479401,0.050167,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976682,0.975741,0.973944,...,0.972517,0.003804,5,0.999326,0.998428,0.999102,1.000000,0.999102,0.999192,0.000504


# Building Machine Learning Classifiers: Explore Gradient Boosting model with grid-search


 **Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

# Explore GradientBoostingClassifier Attributes & Hyperparameters

In [52]:
from sklearn.ensemble import GradientBoostingClassifier

In [53]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 's

# Build our own Grid-search


In [54]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [55]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.872
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.971 / Recall: 0.699 / Accuracy: 0.959
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.921 / Recall: 0.811 / Accuracy: 0.967
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.014 / Accuracy: 0.873
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.934 / Recall: 0.797 / Accuracy: 0.967
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.901 / Recall: 0.825 / Accuracy: 0.966
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.873
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.913 / Recall: 0.804 / Accuracy: 0.965
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.929 / Recall: 0.818 / Accuracy: 0.969
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.871
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.906 / Recall: 0.804 / Accuracy: 0.964
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.908 / Recall: 0.825 / Accura

In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [150], 
    'max_depth': [15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=3, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
result=pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]
print(result)